In [1]:
#if haven't installed ee package, please run the command in next line first
#conda install -c conda-forge earthengine-api

#keep up to date
#conda update -c conda-forge earthengine-api

#install folium package for interactive map
#conda install folium

import ee
import folium

#need to generate token once you authorised
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AdQt8qi-O8hKqBcruyYPJCyq4f0xMBibEXAgQgjmmrc4LlrEfJsVf84HdIw

Successfully saved authorization token.


In [2]:
#load classifier as ee feature collection
#cart_classifier = ee.FeatureCollection("users/ee-gupei/randomforest_classifier_3")
cart_classifier = ee.FeatureCollection("projects/ee-gupei/assets/randomforest_classifier_3")


geometry = ee.FeatureCollection("projects/ee-gupei/assets/Final_data_9k")

In [3]:
geometry

In [4]:
cart_classifier.first()

In [5]:
#sentinel 2A
Sentinel2A = ee.ImageCollection("COPERNICUS/S2_SR");

In [6]:
classifier_string = cart_classifier.first().get('classifier');

In [7]:
#call ee classifier decisionTree
classifier = ee.Classifier.decisionTree(classifier_string);

In [8]:
#set the bands
BANDS = ['B2', 'B3', 'B4', 'B8'];

In [9]:
#filter by date range, and Bands
ic = Sentinel2A.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 15))
ic = ic.select(BANDS).filterDate('2016-07-01', '2020-12-01').median();

In [10]:
classified = ic.classify(classifier);

In [11]:
classified.clip(geometry)

In [19]:
palette = [
  '0000FF', #Water
  '008000', #Veg
  'A52A2A'  #Land
]

# image_vis_params = {
#     'bands': ['B2', 'B3', 'B4', 'B8'],
#     'min': 0,
#     'max': 0.5,
#     'gamma': [0.95, 1.1, 1]
# }


In [21]:
lat = -37.8
lon = 145.2
Map = folium.Map(location=[lat, lon], zoom_start=10)

def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)
    
folium.Map.add_ee_layer = add_ee_layer


In [ ]:
# Add the image layer to the map and display it.
Map.add_ee_layer(classified.clip(geometry), {'palette': palette, 'min': 0, 'max': 2}, 'Classification CART')
display(Map)